In [1]:
from datasets.dataset import *
from models.meta import Meta
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import timm
from tensorboardX import SummaryWriter
import numpy as np
import pandas as pd
import os
import sys
import shutil
from PIL import Image
from models.basenet import *
from utils import *
from configs.config_setting_baseline import setting_config
from copy import deepcopy
import sklearn.metrics as metrics
from torch.cuda.amp import autocast, GradScaler
import torch.nn.init as init
import segmentation_models_pytorch as smp

import warnings
warnings.filterwarnings("ignore")

config = setting_config

In [2]:
def preprocess_batch(batch):
    support_images = batch['support_images'].squeeze(0)
    support_masks = batch['support_masks'].squeeze(0)
    query_images = batch['query_images'].squeeze(0)
    query_masks = batch['query_masks'].squeeze(0)
    return support_images, support_masks, query_images, query_masks

# the function of copying the images
def copy_file_to_folder(source_file, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    dest_path = os.path.join(dest_folder, os.path.basename(source_file))
    shutil.copy(source_file, dest_path)

def evaluation_api(predicted_list,groudtruth_list):
    pre = np.array([item for sublist in predicted_list for item in sublist]).reshape(-1)
    gts = np.array([item for sublist in groudtruth_list for item in sublist]).reshape(-1)
    # confusion_matrix = metrics.confusion_matrix(gts,pre)
    # TN, FP, FN, TP = confusion[0,0], confusion[0,1], confusion[1,0], confusion[1,1] 
    dice = metrics.f1_score(gts,pre)

    return dice

def evaluation_epoch(predicted_list,groundtruth_list):
    TP = [0]*config.num_classes
    FP = [0]*config.num_classes
    FN = [0]*config.num_classes
    dice = [0.0]*config.num_classes
    
    for i in range(len(predicted_list)):
        preds = np.array(predicted_list[i]).reshape(-1)
        gts = np.array(groundtruth_list[i]).reshape(-1)
        for j in range(len(preds)):
            if preds[j] == gts[j]:
                TP[gts[j]] += 1
            else:
                FP[preds[j]] += 1
                FN[gts[j]] += 1        
    
    for i in range(config.num_classes):
        dice[i] = (2 * TP[i])/(FP[i]+FN[i]+2*TP[i]+1)

    mdice = (2*np.sum(TP))/(np.sum(FP)+np.sum(FN)+2*np.sum(TP)+1)    
    return dice,mdice

def evaluation_basenet(base_net,query_images,query_masks,criterion):
    predicted = base_net(query_images)
    loss = criterion(predicted,query_masks)
    predicted = torch.argmax(predicted,dim=1).long()
    predict_numpy = predicted.detach().cpu().numpy().reshape(-1)
    masks_numpy = query_masks.long().detach().cpu().numpy().reshape(-1)
    accuracy = metrics.accuracy_score(masks_numpy,predict_numpy)
    f1_score = metrics.f1_score(masks_numpy,predict_numpy,average=None)
    return accuracy,f1_score,loss

def initialize_weights_he(model):
    for param in model.parameters():
        init.kaiming_uniform_(param, mode='fan_in', nonlinearity='relu')

def initialize_weights_xavier(model):
    for param in model.parameters():
        init.xavier_uniform_(param)

def initialize_weights_normal(model):
    for param in model.parameters():
        init.normal_(param, mean=0, std=1)

def remove_exsits_folder(folderpath):
    if os.path.exists(folderpath):
        shutil.rmtree(folderpath)


In [3]:
print('#----------Creating logger----------#')
sys.path.append(config.work_dir + '/')
log_dir = os.path.join(config.work_dir, 'log')
checkpoint_dir = os.path.join(config.work_dir, 'checkpoints')
resume_model = os.path.join(checkpoint_dir, 'latest.pth')
outputs = os.path.join(config.work_dir, 'outputs')
csv_save = os.path.join(config.work_dir, 'csv')
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(outputs):
    os.makedirs(outputs)
if not os.path.exists(csv_save):
    os.makedirs(csv_save)

global logger
logger = get_logger('test', log_dir)
global writer
writer = SummaryWriter(config.work_dir + 'summary')

log_config_info(config, logger)

#----------Creating logger----------#


In [4]:
print('#----------GPU init----------#')
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu_id
set_seed(config.seed)
device = torch.device('cuda')
torch.cuda.empty_cache()

#----------GPU init----------#


In [5]:
print('#----------Prepareing Datasets----------#')
# create the dataset and dataloader
batch_size = config.batch_size
train_dataset = HAMALL_datasets(config, train=True)
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, num_workers=config.num_workers)
val_dataset = HAMALL_datasets(config, train=False,val=True)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, num_workers=config.num_workers)
test_dataset = HAMALL_datasets(config, train=False)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, num_workers=config.num_workers)
print("trian_dataset length:",len(train_dataset))
print("val_dataset length:",len(val_dataset))
print("test_dataset length:",len(test_dataset))

#----------Prepareing Datasets----------#
trian_dataset length: 900
val_dataset length: 546
test_dataset length: 546


In [6]:
print('#----------Prepareing Model----------#')
in_channels = config.in_channels
out_channels = config.out_channels
base_net = smp.Unet(encoder_name='resnet34', encoder_depth=5, encoder_weights=None, decoder_use_batchnorm=True, decoder_channels=(256, 128, 64, 32, 16), decoder_attention_type=None, in_channels=3, classes=config.out_channels, activation=None, aux_params=None)
# base_net = smp.UnetPlusPlus(encoder_name='resnet34', encoder_depth=5, encoder_weights=None, decoder_use_batchnorm=True, decoder_channels=(256, 128, 64, 32, 16), decoder_attention_type=None, in_channels=3, classes=config.out_channels, activation=None, aux_params=None)
# initialize_weights_he(base_net)
base_net = base_net.to(device)

#----------Prepareing Model----------#


In [7]:
print('#----------Prepareing loss, opt, sch and amp----------#')
criterion = nn.BCEWithLogitsLoss()
meta_optimizer = get_optimizer(config, base_net)
meta_scheduler = get_scheduler(config, meta_optimizer)
criterion = criterion.to(device)

#----------Prepareing loss, opt, sch and amp----------#


In [8]:
print('#----------Set other params----------#')
min_loss = 999
start_epoch = 1
min_epoch = 1
threshold = 0.5

#----------Set other params----------#


In [9]:
print('#----------Start training----------#')
torch.cuda.empty_cache()
info = "%d-resizeh, %d-resizew, %f-outer_lr"%(config.resize_h,config.resize_w,config.outer_lr)
print(info)
logger.info(info)
best_dice_val = 0.0
best_dice_test = 0.0
train_csv = os.path.join(csv_save,"train.csv")
val_csv = os.path.join(csv_save,"val.csv")
test_csv = os.path.join(csv_save,"test.csv")
train_columns = ['Epoch','Loss',"Mdice"]
train_df = pd.DataFrame(columns=train_columns)
val_columns = ['Epoch','Mdice']
val_df = pd.DataFrame(columns=val_columns)
test_columns = ['Epoch','Mdice']
test_df = pd.DataFrame(columns=test_columns)
for epoch in range(start_epoch, config.epoch_num+1):
    # train part
    torch.cuda.empty_cache()
    predicted_list = []
    groundtruth_list = []
    loss_list = []    
    base_net.train()
    for image,mask in train_loader:
        # claer the meta_optimizer, setting zero
        meta_optimizer.zero_grad()
        image = image.to(device)
        mask = mask.to(device)
        image = torch.squeeze(image,dim=1)      # torch.Size([bs, 3, 512, 512])
        mask = torch.squeeze(mask,dim=1)     # torch.Size([bs, 1, 512, 512])
        mask = torch.squeeze(mask,dim=1).float()     # torch.Size([bs, 512, 512])
        predicted = base_net(image)     # torch.Size([bs,out_channels=1,512,512])
        predicted = predicted.squeeze(1)    # torch.Size([bs,512,512])
        loss = criterion(predicted,mask)
        loss.backward()
        meta_optimizer.step()
        loss_list.append(loss.cpu().detach().numpy())
        predicted = (predicted > threshold).long()
        temp_predicted = predicted.cpu().detach().numpy()       # threshold alternative
        predicted_list.append(temp_predicted)
        groundtruth_list.append(mask.long().cpu().detach().numpy())
    # train_dice,train_mdice = evaluation_epoch(predicted_list,groundtruth_list)
    train_dice = evaluation_api(predicted_list,groundtruth_list)
    train_mloss = np.mean(loss_list)
    log_train = f'epoch: {epoch}, loss: {train_mloss}, dice: {train_dice}'
    print("#Train# ",log_train)
    temp_result = pd.Series([epoch,train_mloss,train_dice],index=train_columns)
    train_df = train_df.append(temp_result, ignore_index=True)
    train_df.to_csv(train_csv, index=False)
    
    # validation part
    torch.cuda.empty_cache()
    predicted_list = []
    groundtruth_list = []
    base_net.eval()
    with torch.no_grad():
        for image,mask in val_loader:
            # claer the meta_optimizer, setting zero
            meta_optimizer.zero_grad()
            image = image.to(device)
            mask = mask.to(device)
            image = torch.squeeze(image,dim=1)      # torch.Size([bs, 3, 512, 512])
            mask = torch.squeeze(mask,dim=1)     # torch.Size([bs, 1, 512, 512])
            mask = torch.squeeze(mask,dim=1).float()     # torch.Size([bs, 512, 512])
            predicted = base_net(image)
            temp_predicted = (predicted > threshold).long().cpu().detach().numpy()        # (20, 128, 128)
            predicted_list.append(temp_predicted)
            groundtruth_list.append(mask.long().cpu().detach().numpy())
        # val_dice,val_mdice = evaluation_epoch(predicted_list,groundtruth_list)
        val_dice = evaluation_api(predicted_list,groundtruth_list)
        log_val = f'epoch: {epoch}, dice: {val_dice}'
        print("#Val# ",log_val)
        temp_result = pd.Series([epoch,val_dice],index=val_columns)
        val_df = val_df.append(temp_result, ignore_index=True)
        val_df.to_csv(val_csv, index=False)
        # logger.info(log_val)

    if val_dice > best_dice_val:
        torch.save(base_net.state_dict(), os.path.join(checkpoint_dir, 'best_val.pth'))
        best_dice_val = val_dice
    torch.cuda.empty_cache()
    
    # test part
    torch.cuda.empty_cache()
    predicted_list = []
    groundtruth_list = []
    base_net.eval()
    with torch.no_grad():
        for image,mask in test_loader:
            # claer the meta_optimizer, setting zero
            meta_optimizer.zero_grad()
            image = image.to(device)
            mask = mask.to(device)
            image = torch.squeeze(image,dim=1)      # torch.Size([bs, 3, 512, 512])
            mask = torch.squeeze(mask,dim=1)     # torch.Size([bs, 1, 512, 512])
            mask = torch.squeeze(mask,dim=1).float()     # torch.Size([bs, 512, 512])
            predicted = base_net(image)
            temp_predicted = (predicted > threshold).long().cpu().detach().numpy()        # (20, 128, 128)
            predicted_list.append(temp_predicted)
            groundtruth_list.append(mask.long().cpu().detach().numpy())
        # test_dice,test_mdice = evaluation_epoch(predicted_list,groundtruth_list)
        test_dice = evaluation_api(predicted_list,groundtruth_list)
        log_test = f'epoch: {epoch}, dice: {test_dice}'
        print("#Test# ",log_test)
        temp_result = pd.Series([epoch,test_dice],index=test_columns)
        test_df = test_df.append(temp_result, ignore_index=True)
        test_df.to_csv(test_csv, index=False)
        logger.info(log_test)

    if test_dice > best_dice_test:
        torch.save(base_net.state_dict(), os.path.join(checkpoint_dir, 'best_test.pth'))
        best_dice_test = test_dice
    torch.cuda.empty_cache()



#----------Start training----------#
128-resizeh, 128-resizew, 0.000100-outer_lr
#Train#  epoch: 1, loss: 0.6088895201683044, dice: 0.6131106248781977
#Val#  epoch: 1, dice: 0.6626652295672113
#Test#  epoch: 1, dice: 0.6816687876800144
#Train#  epoch: 2, loss: 0.46793049573898315, dice: 0.7169476172826484
#Val#  epoch: 2, dice: 0.6903764031287335
#Test#  epoch: 2, dice: 0.6774759883309243
#Train#  epoch: 3, loss: 0.4144618809223175, dice: 0.7616970119930201
#Val#  epoch: 3, dice: 0.6701026418648544
#Test#  epoch: 3, dice: 0.6235649726331698
#Train#  epoch: 4, loss: 0.3680786192417145, dice: 0.8099765705879147
#Val#  epoch: 4, dice: 0.7625660041080887
#Test#  epoch: 4, dice: 0.7570327790362006
#Train#  epoch: 5, loss: 0.3456106185913086, dice: 0.8244840877984725
#Val#  epoch: 5, dice: 0.6632573186926352
#Test#  epoch: 5, dice: 0.6392283446593682
#Train#  epoch: 6, loss: 0.31706714630126953, dice: 0.8462579911245729
#Val#  epoch: 6, dice: 0.33261880272672895
#Test#  epoch: 6, dice: 0.339

In [ ]:
best_result_test = "Best dice in testset:" + str(best_dice_test)
print(best_result_test)
logger.info(best_result_test)

Best dice in testset:0.7995751834376551
